arla's stuff

personal pronouns?

The `subscribe` variable type needs to be converted from the logical to the factor type to be used as label for KNN classification––this can be done using `fct_recode()`. Only 2 observations were removed when observations with `NA` were removed, which should not have a drastic effect on our data. 

There are 142 individuals subscribed to the game's newsletter, and 52 who are not; the classes are not very balanced. Thus, upsampling should be performed during pre-processing to minimize the impact of unbalanced classes on classification.

In [22]:
library(tidyverse)
library(tidymodels)
library(cowplot)
library(repr)
library(themis)

set.seed(47)
options(repr.plot.width = 14)

players_url <- "https://raw.githubusercontent.com/oo74/DSCI-100-Project/d932a95bab3bbe9a443dcba02939882b0735483f/data/players.csv"
players <- read_csv(players_url) |>
    mutate(subscribe = fct_recode(as_factor(subscribe), Yes = "TRUE", No = "FALSE"))

nrow(players) - nrow(drop_na(players))
players <- drop_na(players)

players |>
    group_by(subscribe) |>
    summarize(count = n())

Rows: 196 Columns: 7
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (4): experience, hashedEmail, name, gender
dbl (2): played_hours, Age
lgl (1): subscribe

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


[1] 2

subscribe,count
<fct>,<int>
No,52
Yes,142


If multiple K's had the same highest accuracy, the lowest K was selected for computational efficiency. 

what the hell does skip = TRUE do

In [40]:
k_vals <- tibble(neighbors = 1:10)

players_split <- initial_split(players, prop = 0.75, strata = subscribe)
players_train <- training(players_split)
players_test <- testing(players_split)

players_recipe <- recipe(subscribe ~ played_hours + Age, data = players_train) |>
    step_upsample(subscribe, over_ratio = 1, skip = FALSE) |>
    step_center(all_predictors()) |>
    step_scale(all_predictors()) 

players_spec <- nearest_neighbor(weight_func = "rectangular", neighbors = tune()) |>
    set_engine("kknn") |>
    set_mode("classification")

vfold_sets <- players_train |>
    vfold_cv(v = 5, strata = subscribe)

k_accuracies <- workflow() |>
    add_recipe(players_recipe) |>
    add_model(players_spec) |>
    tune_grid(resamples = vfold_sets, grid = k_vals) |>
    collect_metrics() |>
    filter(.metric == "accuracy") |>
    mutate(accuracy = mean) |>
    select(neighbors, accuracy)
    
k_accuracies_plot <- k_accuracies |>
    ggplot(aes(x = neighbors, y = accuracy)) +
    geom_point() +
    geom_line() +
    labs(x = "Neighbors (K)", y = "Accuracy", title = "Fig. 1: Accuracies associated with various K values.") +
    theme(text = element_text(size = 12))

best_k <- k_accuracies |>
    slice_max(accuracy) |>
    slice_min(neighbors) |>
    pull(neighbors)

k_accuracies_plot
best_k

→ A | error:   Some assessment set rows are not available at prediction time. Consider using `skip = TRUE` on any recipe steps that remove rows to avoid calling them on the assessment set.

There were issues with some computations   A: x1

There were issues with some computations   A: x5



Warning message:
“All models failed. Run `show_notes(.Last.tune.result)` for more information.”


ERROR: [1m[33mError[39m in `estimate_tune_results()`:[22m
[33m![39m All models failed. Run `show_notes(.Last.tune.result)` for more information.


In [41]:

# -------------------------



players_best_spec <- nearest_neighbor(weight_func = "rectangular", neighbors = best_k) |>
    set_engine("kknn") |>
    set_mode("classification")

players_fit <- workflow() |>
    add_recipe(players_recipe) |>
    add_model(players_best_spec) |>
    fit(data = players_train)

players_predicted <- players_fit |>
    predict(players_test) |>
    bind_cols(players_test) 

players_predicted_accuracy <- players_predicted |>
    metrics(truth = subscribe, estimate = .pred_class) |>
    filter(.metric == "accuracy") |>
    slice_max(.estimate) |>
    pull(.estimate)

players_predicted_conf <- players_predicted |>
    conf_mat(truth = subscribe, estimate = .pred_class)


players_plot <- players_predicted |>
    ggplot(aes(x = Age, y = played_hours, color = subscribe)) +
    geom_point(alpha = 0.7) +
    labs(x = "Age (years)", y = "Played Hours", title = "Fig. 2: Age, played hours, and actual subscription status of players.", color = "Subscribed?") +
    ylim(0, 5) +
    theme(text = element_text(size = 12), legend.position = "bottom", legend.direction = "horizontal")

players_predicted_plot <- players_predicted |>
    ggplot(aes(x = Age, y = played_hours, color = .pred_class)) +
    geom_point(alpha = 0.7) +
    labs(x = "Age (years)", y = "Played Hours", title = "Fig. 3: Age, played Hours, and predicted subscription status of players.", color = "Predicted to subscribe?") +
    ylim(0, 5) +
    theme(text = element_text(size = 12), legend.position = "bottom", legend.direction = "horizontal")


k_accuracies_plot
players_predicted_conf
plot_grid(players_plot, players_predicted_plot, ncol = 2)


#for billy: is it ok that i condensed this all?

ERROR: [1m[33mError[39m in `step_upsample()`:[22m
[1m[22m[33m![39m The following required column is missing from `new_data` in step
  'upsample_uedxo': subscribe.


In [ ]:
mention why there is third color (opacity)

analyze accuracy, precision, and recall ––what it implies
talk about class balance.

why lim at ylim

tends to predict subscribed because of class imbalance. - high recall, so minimizes false positives.